# Overview
Regulations.gov hosts the publicly submitted comments that are part of Sec. Zinke's monuments review. You can browse the comments there, and download a CSV that lists the comment numbers. This project scrapes these comments into a database (see: [setup.ipynb]) using Selenium Webdriver.

Unfortuantely, the CSV download only lists the most recent 103,000 comments. Regulations.gov confirms it's a limitation on their end. With over 390,000 comments available, a new method is needed. This notebook takes advantage of the fact that the posted comments are numbered sequentially.

# Instructions


## Activate Source Environment
```bash
source activate benm
jupyter notebook
```

## Download Comments by Number
1. The full list of comments is provided at [https://www.regulations.gov/docketBrowser?rpp=25&so=DESC&sb=commentDueDate&po=0&dct=PS&D=DOI-2017-0002]
2. Open the URL above and update MAX_COMMENT_NUMBER below with the the number of results listed (e.g. 390,376 Results) 
3. Run the cells below

In [ ]:
# start here!

# Update the constant below with the number of results listed on regulations.gov

MAX_COMMENT_NUMBER = 390376

In [ ]:
# some helper functions

# to do: download attachments

import psycopg2
import os, errno, csv

from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from dateutil.parser import parse
from datetime import date

ignore_list = []

TIMEOUT = 15 # WebDriver timeout (page load, etc.), in seconds

def benm_driver():
    fp = webdriver.FirefoxProfile()
    fp.set_preference("http.response.timeout", TIMEOUT)
    fp.set_preference("dom.max_script_run_time", TIMEOUT)
    driver = webdriver.Firefox(firefox_profile=fp)
    driver.implicitly_wait(TIMEOUT) # seconds
    return driver

def get_comments(comments):
    conn = psycopg2.connect("dbname=benm user=postgres")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    query = 'INSERT INTO comments (document_id, tracking_number, date_posted, comment, has_attachments, retrieved) VALUES (%(document_id)s, %(tracking_number)s, %(date_posted)s, %(comment_text)s, %(has_attachments)s, now());'
    ignore_query = "INSERT INTO ignore_list (document_id, reason) VALUES (%s, %s);"

    driver = benm_driver()
    
    for comment in comments:
        document_id = 'DOI-2017-0002-' + '{0:04d}'.format(comment) # pad numbers less than 1000 with leading zeros
        comment_values = get_comment(driver, document_id)
        if comment_values:
            try:
                cur.execute(query, comment_values)
            except Exception as ex:
                if ex.pgcode == '23505': # unique constraint violated
                    cur.execute(ignore_query, (document_id, 'Duplicate Tracking Number'))
                    print('Duplicate tracking number found for comment %s' % document_id)
                else:
                    print(ex)
                    print('consider adding %s to ignore list' % document_id)
                    raise
        else:
            print("Error retrieving document %s." % document_id)
            cur.execute(ignore_query, (document_id, 'unknown error'))
            driver.quit()
            driver = benm_driver()

    driver.quit()
    
    # conn.commit() # Make the changes to the database persistent. Not used if autocommit=True
    conn.close()
    
def get_comment(driver, document_id):
    try:
        url = 'https://www.regulations.gov/document?D=' + document_id
        driver.get(url)
        element = WebDriverWait(driver, TIMEOUT).until(
            EC.title_is("Regulations.gov - Comment")
        )

        element = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[3]/div/table/tbody/tr/td[3]/div/div/div[2]/div[1]/div[1]/span[2]')
        source_document_id = element.text
        assert document_id == source_document_id # make sure the page matches
        element = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[3]/div/table/tbody/tr/td[3]/div/div/div[2]/div[1]/div[2]/span[2]')
        tracking_number = element.text
        element = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[3]/div/table/tbody/tr/td[3]/div/div/div[2]/div[4]/div/div/span[2]')
        d = parse(element.text)
        date_posted = date(d.year, d.month, d.day)
        element = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[3]/div/table/tbody/tr/td[1]/div/div[3]/div[1]/div/div[2]')
        comment_text = element.text
        try:
            element = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[3]/div/table/tbody/tr/td[1]/div/div[3]/div[2]/div[1]/h2/span')
            has_attachments = (element.text == "Attachments")
        except NoSuchElementException:
            has_attachments = False

        result = { 
            'document_id': document_id,
            'tracking_number': tracking_number,
            'date_posted': date_posted,
            'comment_text': comment_text,
            'has_attachments': has_attachments
        }

        return result
    
    except TimeoutException as ex:
        return False
    except NoSuchElementException as ex:
        return False
    else:
        return False

In [ ]:
ids = range(1000, MAX_COMMENT_NUMBER)

# get the set of IDs of comments we've already downloaded, and the ignore list
with psycopg2.connect("dbname=benm user=postgres") as conn:
    with conn.cursor() as cur:
        try:
            cur.execute('SELECT substring(document_id from 15)::int FROM comments ORDER BY substring(document_id from 15)::int;')
            downloaded_comments = set([c[0] for c in cur.fetchall()])
            
            cur.execute('SELECT substring(document_id from 15)::int FROM ignore_list;')
            ignore_list = set([c[0] for c in cur.fetchall()])
        except psycopg2.Error as e:
            print (query)
            print (e.pgerror)
conn.close()

# intersect the range of IDs and the set of downloaded/ignore comments, and 
# take the difference as our working list
dl_list = set(ids).difference(downloaded_comments.union(ignore_list))
dl_list = list(dl_list)
print('comments in db: ' + str(len(downloaded_comments)))
print('ignore list: ' + str(len(ignore_list)))
print('to download: ' + str(len(dl_list)))

In [ ]:
# spawn workers to scrape batch_size comments each

from multiprocessing import Process

workers = 4 # 16
batch_size = int(len(dl_list) / workers)

processes = []

for i in range(workers):
    processes.append(Process(target=get_comments, args=(dl_list[i*batch_size:((i+1)*batch_size)-1],)))
    processes[i].start()

for i in range(workers):
    processes[i].join()

In [ ]:
import pandas
import pandas.io.sql as psql

# write comments to CSV
conn = psycopg2.connect("dbname=benm user=postgres")
df = psql.read_sql("SELECT document_id, tracking_number, date_posted, retrieved, has_attachments, comment FROM comments;", conn)
df[['document_url']] = 'https://www.regulations.gov/document?D=' + df[['document_id']]

# sort rows by document id, then drop the internal database id column
df['id'] = df['document_id'].str[14:]
df = df.sort_values(by='id')
df = df.drop('id', 1) # 1 is the axis number, 0 for rows, 1 for columns
df.to_csv('dataset/comments.csv', index=False)

# write ignore list to CSV
df = psql.read_sql("SELECT * FROM ignore_list;", conn)
df[['document_url']] = 'https://www.regulations.gov/document?D=' + df[['document_id']]

# sort rows by document id, then drop the internal database id column
df['id'] = df['document_id'].str[14:]
df = df.sort_values(by='id')
df = df.drop('id', 1) # 1 is the axis number, 0 for rows, 1 for columns
df.to_csv('dataset/ignore_list.csv', index=False)


In [ ]:
# to do - automate packaging?
# to package - create xlsx version, save as zip